# Convolutional Neural Network

### Importing the libraries

In [ ]:
!pip install --upgrade tensorflow keras Pillow

In [1]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [2]:
tf.__version__

'2.3.1'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [3]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 8000 images belonging to 2 classes.


### Preprocessing the Test set

In [65]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 2000 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [89]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [90]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

### Step 2 - Pooling

In [91]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [92]:
cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a third convolutional layer

In [93]:
cnn.add(tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a fourfh convolutional layer

In [94]:
# cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', kernel_initializer='he_uniform', padding='same'))
# cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [95]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [96]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

### Step 5 - Output Layer

In [97]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [98]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [102]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 25)

Epoch 1/25
250/250 [==============================] - 125s 498ms/step - loss: 0.5150 - accuracy: 0.7452 - val_loss: 0.4995 - val_accuracy: 0.7550
Epoch 2/25
250/250 [==============================] - 81s 325ms/step - loss: 0.4830 - accuracy: 0.7640 - val_loss: 0.4557 - val_accuracy: 0.7860
Epoch 3/25
250/250 [==============================] - 71s 284ms/step - loss: 0.4687 - accuracy: 0.7760 - val_loss: 0.4389 - val_accuracy: 0.7985
Epoch 4/25
250/250 [==============================] - 72s 288ms/step - loss: 0.4440 - accuracy: 0.7887 - val_loss: 0.4472 - val_accuracy: 0.7955
Epoch 5/25
250/250 [==============================] - 63s 251ms/step - loss: 0.4146 - accuracy: 0.8056 - val_loss: 0.4669 - val_accuracy: 0.7805
Epoch 6/25
250/250 [==============================] - 56s 223ms/step - loss: 0.4120 - accuracy: 0.8114 - val_loss: 0.4217 - val_accuracy: 0.8085
Epoch 7/25
250/250 [==============================] - 53s 210ms/step - loss: 0.3903 - accuracy: 0.8236 - val_loss: 0.4296 - val_a

## Part 4 - Making a single prediction

In [ ]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('dataset/single_prediction/cat_or_dog_1.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
  prediction = 'dog'
else:
  prediction = 'cat'

In [ ]:
print(prediction)

## Fim